# Inbound Sales Dashboard Sample Data python script
Purpose:
Generate randomized data for a portfolio dashboard that is similar to real dashboards I have built for corporations in Inbound Sales reporting.

Steps:
1) Generate sales agent employee data
Inbound sales reporting focused primarily on sales agents' performance and calls and sales are both handled by agents.
2) Generate fake handled phone call data
This dashboard will be focusing on sales agents so we will not be looking at service level, missed calls, etc.  So I will be looking at the employee start dates and then generating calls after their start date for each employee.  This call data will have a handled by agent field tying back to the employee and a date field tying to the date dimension.
4) Generate fake sales data
In inbound sales, a sale can only be made when a call is handled.  So I will start with the phone call data and then create sales based on the phone call.  Not every phone call results in a sale, so need to use a randomized probability a sale was made.  The sales data will have a handled by agent field tying back to the employee and a date field tying to the date dimension.

Not Included:
To focus the reporting we will not be looking at things like service level, missed calls, call text, call skills, orders, product tiers, commission, etc.  Things like those were usually reported on in specialized sales operations dashboards.  Inbound sales dashboards were primarily concerned with sales performance by agents and leader teams.

In [1]:
#import needed libraries
import pandas as pd #dataframes
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta #job titles/sales based on tenure
import numpy as np #randoms on arrays, etc.


from faker import Faker #randomized fake data
fake = Faker() #create a usable faker instance

In [2]:
#1) generate sales agent employee data

#create a name function so all names are formatted the same + reduce modularity
def create_random_name():
    return fake.last_name() + ", " + fake.first_name()  


#create the dataframe that will hold the data prior to CSV export.
employee_columns = ['Employee_ID','Employee_Name', 'Job_Title', 'Hire_Date', 'Call_Center','Manager_Name','Supervisor_Name']


hire_date_start = date(2025-70,1,1)
hire_date_end = date(2025,1,1) 



#create a list of call centers. Each manager reports to 1 call center.
centers_data = {
    'Call_Center': ['Narshe','Zanarkand','Cornelia','Nibelheim','Costa del Sol','Midgar','Zozo','Daguerreo','Dali','Albrook']
}
centers_df = pd.DataFrame(centers_data)




#create a list of managers. Each supervisor reports to 1 manager.
managers_df = pd.DataFrame(columns=employee_columns)
managers_count = 50


#generate realistic managers data (using the call center list)
for i in range(managers_count):
    employee_id = i    
    employee_name = create_random_name()
    job_title = 'Manager, Inbound Sales'
    hire_date = fake.date_time_between(start_date=hire_date_start, end_date=hire_date_end ).date()
    call_center_name = centers_df['Call_Center'].sample(n=1).iloc[0] #randomly pick call center for this manager
    manager_name = '' #higher hierarchy not needed for this.
    supervisor_name = ''        
    
    managers_df.loc[len(managers_df)] = [employee_id, employee_name, job_title, hire_date, call_center_name, manager_name, supervisor_name] #add new row with this data

managers_df.head()



,Employee_ID,Employee_Name,Job_Title,Hire_Date,Call_Center,Manager_Name,Supervisor_Name
0,0,"Bell, Thomas","Manager, Inbound Sales",2011-07-31,Cornelia,,
1,1,"Rogers, Richard","Manager, Inbound Sales",1973-10-21,Narshe,,
2,2,"Miller, Jose","Manager, Inbound Sales",2024-11-01,Daguerreo,,
3,3,"Perez, Tony","Manager, Inbound Sales",2001-06-03,Narshe,,
4,4,"Mitchell, Kelsey","Manager, Inbound Sales",2002-08-10,Dali,,


In [3]:
#generate realistic supervisors data (using the managers list)

#create a list of supervisors. Each supervisor reports to 1 manager.
supervisors_df = pd.DataFrame(columns=employee_columns)
supervisors_count = managers_count * 5

for i in range(supervisors_count):
    employee_id = i + managers_count + 1   
    employee_name = create_random_name()
    job_title = 'Supervisor, Inbound Sales'
    hire_date = fake.date_time_between(start_date=hire_date_start, end_date=hire_date_end ).date()

    manager_row = managers_df.sample(n=1) #pick one random manager to assign to this sup
    
    call_center_name = manager_row['Call_Center'].item() #use the manager's call center so it matches the hierarchy
    manager_name = manager_row['Employee_Name'].item()
    supervisor_name = ''        
    
    supervisors_df.loc[len(supervisors_df)] = [employee_id, employee_name, job_title, hire_date, call_center_name, manager_name, supervisor_name] #add new row with this data

supervisors_df.head()


,Employee_ID,Employee_Name,Job_Title,Hire_Date,Call_Center,Manager_Name,Supervisor_Name
0,51,"Johnson, John","Supervisor, Inbound Sales",2002-08-24,Daguerreo,"Carson, Julie",
1,52,"Neal, Destiny","Supervisor, Inbound Sales",1968-08-02,Daguerreo,"Kemp, Deanna",
2,53,"Quinn, Corey","Supervisor, Inbound Sales",1982-02-22,Cornelia,"Lewis, Stephen",
3,54,"Hughes, Danielle","Supervisor, Inbound Sales",1991-01-24,Albrook,"Gill, Taylor",
4,55,"Miller, Todd","Supervisor, Inbound Sales",1979-03-30,Daguerreo,"Carson, Julie",


In [4]:
#generate realistic employee data (using the supervisors list)

#create a list of agents. Each agent reports to 1 supervisor
agents_df = pd.DataFrame(columns=employee_columns)
agents_df['Tenure_Years'] = np.nan

agents_count = supervisors_count * 10

for i in range(agents_count):
    employee_id = i + managers_count + supervisors_count + 10   
    employee_name = create_random_name()
    
    hire_date = fake.date_time_between(start_date=hire_date_start, end_date=hire_date_end ).date()
    
    #increase job title & sales metrics based on tenure
    years = (relativedelta(date.today(), hire_date)).years
    
    if years <= 1:
        job_title = 'Agent I, Inbound Sales'
    elif years <= 3:
        job_title = 'Agent II, Inbound Sales'
    elif years <= 5:
        job_title = 'Agent III, Inbound Sales'
    else:
        job_title = 'Agent IV, Inbound Sales'

    supervisor_row = supervisors_df.sample(n=1) #pick one random manager to assign to this sup
    
    call_center_name = supervisor_row['Call_Center'].item() #use the manager's call center so it matches the hierarchy
    manager_name = supervisor_row['Manager_Name'].item()
    supervisor_name = supervisor_row['Employee_Name'].item()
    
    agents_df.loc[len(agents_df)] = [employee_id, employee_name, job_title, hire_date, call_center_name, manager_name, supervisor_name, years] #add new row with this data

agents_df.head()



,Employee_ID,Employee_Name,Job_Title,Hire_Date,Call_Center,Manager_Name,Supervisor_Name,Tenure_Years
0,310,"Strickland, Maria","Agent IV, Inbound Sales",2010-01-13,Narshe,"Rogers, Richard","Archer, Julie",15
1,311,"Hooper, Virginia","Agent IV, Inbound Sales",1999-09-13,Daguerreo,"Hanna, Leslie","Jones, Barbara",25
2,312,"Thompson, Christian","Agent IV, Inbound Sales",1993-03-27,Nibelheim,"Dennis, Christopher","Houston, Natalie",32
3,313,"Hernandez, Ashley","Agent IV, Inbound Sales",1957-02-03,Midgar,"Perkins, Mary","Williams, Sarah",68
4,314,"Harper, Jordan","Agent IV, Inbound Sales",1977-09-26,Daguerreo,"Martinez, Brian","Hall, Shannon",47


In [5]:
agent_title_test_df = agents_df[agents_df['Job_Title'] == 'Agent I, Inbound Sales']
agent_title_test_df.head()

,Employee_ID,Employee_Name,Job_Title,Hire_Date,Call_Center,Manager_Name,Supervisor_Name,Tenure_Years
49,359,"Patterson, David","Agent I, Inbound Sales",2024-09-11,Costa del Sol,"Johnson, Kevin","Davidson, Anna",0
78,388,"Spears, Mike","Agent I, Inbound Sales",2024-11-28,Daguerreo,"Nash, Lisa","Yu, Meghan",0
237,547,"Miller, Stefanie","Agent I, Inbound Sales",2024-07-02,Costa del Sol,"Rhodes, Amber","Adams, Stacy",1
250,560,"Austin, Joshua","Agent I, Inbound Sales",2024-04-15,Dali,"Rivera, Jordan","Rodriguez, Michael",1
287,597,"Morales, Frank","Agent I, Inbound Sales",2024-01-31,Zanarkand,"West, John","Johnson, Matthew",1


In [6]:
agents_df.to_csv('inbound sales dashboard - dim agents.csv', index=False) #export to CSV for use in the dashboard

In [12]:
#generate fake handled phone call data based on the agent row & their start date.  (calls can only be handled by agents)

#create a "cross join" of dates and employees
first_date = date(2015,1,1) #make sure we don't pull 70 years of call data!
calendar_key = pd.date_range(start=first_date, end=date.today(), name='Call_Date')
agent_ids = agents_df['Employee_ID'].unique() #don't add the other fields to the multi index

calendar_agents_multi_index = pd.MultiIndex.from_product(
    [calendar_key, agent_ids]
    ,names=['Call_Date','Employee_ID']
)
calendar_agents_df = pd.DataFrame(index=calendar_agents_multi_index).reset_index() #reset index makes the multi index into a normal dataframe

#add in hire date so we can remove days before they were hired
calendar_agents_df = pd.merge(
    calendar_agents_df #left table
    , agents_df[['Employee_ID','Hire_Date','Tenure_Years']] #right table
    , on='Employee_ID' #join
    , how='left' #join type
)

#remove rows with days before hire date as potential call dates
calendar_agents_df = calendar_agents_df[
    calendar_agents_df['Call_Date'] >= calendar_agents_df['Hire_Date']
].copy() #copy guarantees a new dataframe and not a "view" reference

#remove random number of days off.  5 out of 7 days + some PTO time.
calendar_agents_df = calendar_agents_df.sample(frac=(4.9 / 7)).reset_index(drop=True) #reset index with drop creates a new sequential index.

#Add in randomized call data using a set-based (vectorized) approach to prevent for loop performance issues with dataframes
rows_count = len(calendar_agents_df)
calendar_agents_df['Sales_Calls'] = np.random.randint(1, 40, size=rows_count)
calendar_agents_df['New_Customer_Calls'] = np.random.randint(1, 31, size=rows_count)
calendar_agents_df['Upgrade_Calls'] = np.random.randint(0, 11, size=rows_count)
calendar_agents_df['Total_Calls'] = 0

#phone status time
calendar_agents_df['ACW_Time'] = np.random.randint(0, 60*60*3, size=rows_count) #account notes & call avoidance
calendar_agents_df['Hold_Time'] = np.random.randint(0, 60*60*2, size=rows_count) #customer on hold
calendar_agents_df['AUX_Time'] = np.random.randint(0, 60*60*4, size=rows_count) #meetings, breaks & call avoidance
calendar_agents_df['Total_Staffed_Time'] = 0

calendar_agents_df['Total_Calls'] = calendar_agents_df[
    ['Sales_Calls','New_Customer_Calls','Upgrade_Calls']
].sum(axis=1) #axis 1 = sum over rows (SQL); axis 0 = sum over columns (Excel)

calendar_agents_df['Total_Staffed_Time'] = calendar_agents_df[
    ['ACW_Time','Hold_Time','AUX_Time']
].sum(axis=1)




calendar_agents_df.head()

,Call_Date,Employee_ID,Hire_Date,Tenure_Years,Sales_Calls,New_Customer_Calls,Upgrade_Calls,Total_Calls,ACW_Time,Hold_Time,AUX_Time,Total_Staffed_Time,RGU_Close_Rate,RGU_Total
0,2022-05-01,952,1983-10-11,41,10,14,2,26,8004,6611,4557,19172,0.125103,3
1,2017-04-10,1670,2012-10-13,12,38,3,5,46,6275,4528,7753,18556,0.405333,18
2,2017-01-18,2476,2002-08-23,23,19,11,4,34,3938,5481,737,10156,0.766963,26
3,2024-06-19,2262,1983-08-27,42,38,22,1,61,9033,3020,11680,23733,0.625040,38
4,2020-06-18,2630,2018-08-04,7,1,29,9,39,3841,5556,2723,12120,0.189313,7


In [16]:
#go ahead and add sales metrics in also as it's dependent on calls.  We can split into separate tables in power query.
calendar_agents_df['RGU_Close_Rate'] = np.random.uniform(low=.10,high=.80, size=rows_count)
calendar_agents_df['RGU_Total'] = np.trunc(calendar_agents_df['RGU_Close_Rate'] * calendar_agents_df['Total_Calls']).astype(int)

calendar_agents_df['Customer_Close_Rate'] = np.random.uniform(low=.10,high=.60, size=rows_count)
calendar_agents_df['Customer_Total'] = np.trunc(calendar_agents_df['Customer_Close_Rate'] * calendar_agents_df['Total_Calls']).astype(int)

calendar_agents_df['Revenue_Per_Call'] = np.random.uniform(low=20, high=60, size=rows_count)
calendar_agents_df['Revenue'] = calendar_agents_df['Revenue_Per_Call'] * calendar_agents_df['Total_Calls']

calendar_agents_df['Call_Quality'] = np.random.uniform(low=.10, high=1, size=rows_count) #QA measurement

calendar_agents_df.head()

,Call_Date,Employee_ID,Hire_Date,Tenure_Years,Sales_Calls,New_Customer_Calls,Upgrade_Calls,Total_Calls,ACW_Time,Hold_Time,AUX_Time,Total_Staffed_Time,RGU_Close_Rate,RGU_Total,Revenue_Per_Call,Revenue,Call_Quality,Customer_Close_Rate,Customer_Total
0,2022-05-01,952,1983-10-11,41,10,14,2,26,8004,6611,4557,19172,0.473322,12,49.838237,1295.794149,0.898000,0.319026,8
1,2017-04-10,1670,2012-10-13,12,38,3,5,46,6275,4528,7753,18556,0.104593,4,39.366833,1810.874317,0.145084,0.283286,13
2,2017-01-18,2476,2002-08-23,23,19,11,4,34,3938,5481,737,10156,0.551853,18,20.369958,692.578562,0.104516,0.424339,14
3,2024-06-19,2262,1983-08-27,42,38,22,1,61,9033,3020,11680,23733,0.297212,18,51.073375,3115.475856,0.228484,0.313170,19
4,2020-06-18,2630,2018-08-04,7,1,29,9,39,3841,5556,2723,12120,0.483831,18,48.458838,1889.894674,0.567941,0.327481,12


In [17]:
calendar_agents_df.to_csv('Inbound Sales Dashboard Call and Sales Data.csv')